In [29]:
from memo import memo, domain
import jax.numpy as np
import jax
from enum import IntEnum
from matplotlib import pyplot as plt
from collections import defaultdict
import pandas as pd

In [ ]:
df_human = pd.read_csv("PD_data.csv")
df_human.head()
# chris behavior category?

,ppt_number,ppt_gender,condition,round_number,binary_classification,chris_prob,ppt_contrib,chris_contrib,ppt_RT,prediction_report,prediction_RT,chris_behavior,turing_test,ppt_payment
0,1,female,1,1,False,0.35,20,20,3743.000,0.09,6695.0,neither,human,1.76
1,1,female,1,2,False,0.35,20,0,1600.005,0.09,6695.0,neither,human,1.76
2,1,female,1,3,False,0.35,0,0,1653.000,0.09,6695.0,neither,human,1.76
3,1,female,1,4,False,0.35,0,20,1711.005,0.09,6695.0,neither,human,1.76
4,1,female,1,5,False,0.35,20,0,1631.000,0.09,6695.0,neither,human,1.76


### Game setup

In [ ]:
# Actions and payoff matrix
actions = [0, 1]  # 0: Cooperate, 1: Defect
payoffs = np.array([[[20, 20], [-20, 40]], 
                    [[40, -20], [0, 0]]])  # Payoff matrix

In [ ]:
@jax.jit
def get_payoff_human(a_1, a_2):
    return payoffs[a_1, a_2, 0]

In [ ]:
@memo
def player_human[a: actions](beta_human=10, cooperate_other=0.5):
    player_human: thinks[
        player_2: chooses(a_2 in actions, wpp=cooperate_other)
    ]
    player_human: chooses(a_1 in actions, 
                    wpp=exp(beta_human * (Pr[player_2.a_2 == a_1] * get_payoff_human(a_1, a_1) + (1-Pr[player_2.a_2 == a_1]) * get_payoff_human(a_1, 1-a_1)))
                    )
    return Pr[player_human.a_1 == a]

### Process data for each play

In [10]:
df_human = df_human.sort_values(by=['ppt_number', 'round_number'])

In [12]:
df_human.head()

,ppt_number,ppt_gender,condition,round_number,binary_classification,chris_prob,ppt_contrib,chris_contrib,ppt_RT,prediction_report,prediction_RT,chris_behavior,turing_test,ppt_payment
0,1,female,1,1,False,0.35,20,20,3743.000,0.09,6695.0,neither,human,1.76
1,1,female,1,2,False,0.35,20,0,1600.005,0.09,6695.0,neither,human,1.76
2,1,female,1,3,False,0.35,0,0,1653.000,0.09,6695.0,neither,human,1.76
3,1,female,1,4,False,0.35,0,20,1711.005,0.09,6695.0,neither,human,1.76
4,1,female,1,5,False,0.35,20,0,1631.000,0.09,6695.0,neither,human,1.76


In [19]:
# Extra play information from human data
def process_player_data(df):
    round_info = {}
    for _, row in df.iterrows():
        round_number = row['round_number']
        chris_prob = row['chris_prob']
        ppt_action = row['ppt_contrib']
        chris_action = row['chris_contrib']

        round_info[round_number] = {
            'chris_prob': chris_prob,
            'ppt_action': ppt_action,
            'chris_action': chris_action}
    return round_info

data_human = df_human.groupby('ppt_number').apply(process_player_data)

In [25]:
data_human

ppt_number
1      {1: {'chris_prob': 0.35, 'ppt_action': 20, 'ch...
3      {1: {'chris_prob': 0.35, 'ppt_action': 20, 'ch...
4      {1: {'chris_prob': 0.35, 'ppt_action': 20, 'ch...
5      {1: {'chris_prob': 0.35, 'ppt_action': 20, 'ch...
6      {1: {'chris_prob': 0.35, 'ppt_action': 20, 'ch...
                             ...                        
152    {1: {'chris_prob': 0.65, 'ppt_action': 20, 'ch...
153    {1: {'chris_prob': 0.65, 'ppt_action': 20, 'ch...
154    {1: {'chris_prob': 0.65, 'ppt_action': 20, 'ch...
155    {1: {'chris_prob': 0.65, 'ppt_action': 20, 'ch...
156    {1: {'chris_prob': 0.65, 'ppt_action': 0, 'chr...
Length: 142, dtype: object

### Simulation

In [ ]:
num_iter = 80
ppts_id = data_human.index.tolist()
# Prior on the other player
cooperate_other = 0.5

# Keep track of actions of the human players
actions_human_simulated = defaultdict(list)
cooperate_human_simulated = defaultdict(list)
actions_human_real = defaultdict(list)

# Do simulation for each player in dataset
for player_id in ppts_id:
    player_data = data_human[player_id]

    # Iterate through rounds
    for iteration in range(num_iter):
        round_data = player_data[iteration+1]

        # Record real action of human player
        action_human_real = 0 if round_data['ppt_action'] == 20 else 1
        actions_human_real[player_id].append(action_human_real)

        # Simulate human player
        human_cooperate_prob = player_human(beta_human=10, cooperate_other=cooperate_other)
        cooperate_human_simulated[player_id].append(human_cooperate_prob)

        # Sample action based on cooperate probability (sample or assign?)
        action_human_simulated = np.random.choice(actions, p=[human_cooperate_prob, 1 - human_cooperate_prob])
        actions_human_simulated[player_id].append(action_human_simulated)

        # Observe action of the other player and update prior
        chris_action = 0 if round_data['chris_action'] == 20 else 1

        # TODO: update cooperate_other based on action_other



In [ ]:
# TODO: Visualization